In [ ]:
model_data = 's3://...'

## Get model artefact and push it to Git repository

In [ ]:
!git clone https://gitlab.com/juliensimon/test-models.git

In [ ]:
%%sh -s $model_data
aws s3 cp $1 .
tar xvfz model.tar.gz -C test-models/model 

In [ ]:
# cd test-models
# git add model
# git commit -m 'New model'
# git push

## Install ecs-cli

In [ ]:
%%sh
curl -o /usr/local/bin/ecs-cli https://amazon-ecs-cli.s3.amazonaws.com/ecs-cli-linux-amd64-latest
chmod 755 /usr/local/bin/ecs-cli

## Create cluster

In [ ]:
%%sh 

# Make sure your IAM role includes ecs:CreateCluster
aws ecs create-cluster --cluster-name fargate-demo
# ecs-cli : https://github.com/aws/amazon-ecs-cli
ecs-cli configure --cluster fargate-demo --region eu-west-1

In [ ]:
%%sh

ecs-cli ps --desired-status RUNNING

## Create log group

In [ ]:
%%sh
# You only need to do this once
aws logs create-log-group --log-group-name awslogs-tf-ecs

## Run inference task

https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [ ]:
%%sh

# You only need to do this when you create and update the task definition
aws ecs register-task-definition --cli-input-json file://inference-fargate-tf230-sagemaker.json

In [ ]:
%%sh

# Update these with your own settings

export SECURITY_GROUP_ID=sg-xxx # SSH + Tensorflow Serving ports

export SUBNET_ID=subnet-xxx              # Any public subnet in your default VPC will do

aws ecs run-task --cluster fargate-demo --task-definition inference-fargate-tf230:2 --count 1 \
    --launch-type FARGATE \
    --network-configuration "awsvpcConfiguration={subnets=[$SUBNET_ID], \
                            securityGroups=[$SECURITY_GROUP_ID], \
                            assignPublicIp=ENABLED}"

In [ ]:
%%sh

ecs-cli ps --desired-status RUNNING

## Predict

In [ ]:
inference_task_ip = 'a.b.c.d'  # Update the IP address 
inference_url = 'http://'+inference_task_ip+':8501/v1/models/1:predict'

In [ ]:
!pip -q install keras

In [ ]:
import keras
import random
from keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

In [ ]:
num_samples = 10
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

data = images.reshape(num_samples, 28, 28, 1)

In [ ]:
import json, requests
import numpy as np

headers = {"content-type": "application/json"}
data    = json.dumps({"signature_name": "serving_default", "instances": data.tolist()})

json_response = requests.post(inference_url, data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']
#print(predictions)
predictions = np.array(predictions).argmax(axis=1)

print("Labels     : ", labels)
print("Predictions: ", predictions)

In [ ]:
%%sh
#aws ecs stop-task --cluster fargate-demo --task arn:aws:ecs:eu-west-1:<TASK_ID>

In [ ]:
%%sh
#ecs-cli down -f --cluster fargate-demo 